# CH 10-11

## TOC<a id='toc'></a>
* [Ch10 Notes](#ch10_notes)
* [Ch11 Notes](#ch11_notes)

### CH10 Notes <a id='ch10_notes'></a>
[toc](#toc)

### Protocols and Duck Typing
* In the context of object oriented programing, a **protocol** is an informal interface, defined only in the documentation and not in the code.
* duck typing: "Dont check whether it *is*-a duck: check whether it *quacks*-like-a duck, *walks*-like-a-duck, etc, etc, depending on exactly what subset of duck-like bahavior you need"

### Slicing
* `obj[1:2:4,9]`  means `__getitem__` receives tuple `(Slice(1,2,4), 9)`
* Nice functionality of Slice: `slice(-3, None, None).indices(5)` --> `(2,5,1)`  -- it translates to *real* index values

<br>
<hr>
<font color=blue> Design hints: Excecive use of `isinstance` if otfen a sign of bad )) design, though sometimes it is justified. In these cases, it is preferable to check against Abstract Base Clases to make API more flexible and future proof. </font>
<br>
<hr>

### Dynamic access
* `__getattr__` method invoked bt interpreter when attribute ookup fails.
    - when `my_0bj.x` python looks to see if `my_obj` instance has x, if not, look at class, if not look at superclasses, if not, call `__getattr__(self, 'x')`
    - it is actually a little more complicated than this; more on this in  a later chapter
* `__setattr__` - what is called when doing `my_obj.x = <val>`.
    - can use it to disable the setting of some attributes
    - often used in tandem with getattr above, because of logic chain of how it is called.
    - this is even called in `__init__`
* similarly `__getitem__` has counterpart `__setitem__`

In [3]:
class myClass:
    def __init__(self,x,y):
        self.x = x
        self.y = y

In [4]:
myObj = myClass(1,2)

In [6]:
myObj.x

1

In [7]:
myObj.x = 5

In [8]:
myObj.x

5

In [9]:
myObj.z = 10

In [10]:
myObj.__dict__.keys()

dict_keys(['x', 'y', 'z'])

In [11]:
class myClass2:
    def __init__(self,x,y):
        self.x = x
        self.y = y
        
    def __setattr__(self, name, value):
        print('Denied')

In [12]:
myObj2 = myClass2(1,2)

Denied
Denied


In [14]:
myObj2.__dict__.keys()

dict_keys([])

### CH11 Notes <a id='ch11_notes'></a>
[toc](#toc)

### Interfaces and Protocols in Python Culture
* protcols ~ informal interfaces that make polymorphism work in languages with dynamic typing
    - independent of inheritance. A class may implement several protocols
* a useful definition of an interface is: the subset of an objects public methods that enable it to play a specific role in the system
* The philosophy of the python data model is to cooperate with essential protocols as much as possible
    - in particular for sequences; iter and contains fall back on `__getitem__`
* **Monkey Patching**: Changing a class of module at runtime, without touching the source code
    - can monkey patch to implement protocol, and onjects/functions that expect protocol won't know the difference [protocols are dynamic]

#### Alex Martelli: Waterfowl and ABCs
* in python, duck typing boils down to avoiding the use of `isinstance` (not to mention the worse approach of checking `type(foo) is bar` because this inhibits even simplest forms of inheritance)
* recomends supplementing *duck typing* with *goose typin*, which means using `isinstance(obj, cls)` is now just fune, as long as `cls` is ab abstract based class.
    - in other words, cls's metaclass is `abc.ABCMeta`
* Most often, you do not need to define your own ABCs
    - less than 1% of the "most advanced pythonistas" ever need to do this
* VIP: Inherting from an ABC is more than implementing the required methods: it's also a clear declaration of intent by the developer.

<br>
<hr>
Only real justification to write one - use it as an extension point in a framework
<hr>
<br>

* Doug Hellman (from PyMOTW):
    - "By defining an abstract base class, a common API can be established for a set of subclasses. This capability is especially useful in situations where someone less familar with the source for an application is going to provide plug-in extensions..."
* David Beazly and Brian Jones (Python Cookbook)
    - "Although ABCs facilitate type checking, it's not something that you shoulf overuse in a program. At its heart, Python is a dynamic language that gives you great flexibility. Trying to enforce type constraints everywhere tends to result in code that is more complicated than it needs to be. You should embrace Python's flexibility."
* Leonardo Rochael (technical reviewer)
    - "If you feel tempted to create a custom ABC, please first try to solve your problem through regular duck typing"

* Claim: Even with abcs, too many if/elif/else block with insinstance checks still smells bad.
    - you should use polymorphism better - design code so that interpreted dispatches appropriate method.
* OTOH, it is usually OK to perform insinstance check against an ABC if you must enforce an API contract.
* Outside of frameworks, duck typing is often simpler and more felxible thatn type checks.
    - use it like a duck and if it fails, there is your check.
* Again from Alex:  "ABCs are meant to encapsulte very general concepts, abstractions, introduced by a framework - things like "a sequence" and "an exact number". [Readers] most likely don't need to write any new ABCs. just existing ones correctly, to get 99.9% of the benefits without serious risk of misdesign.

### Sublcassing ABCs
* many useful abstract classes exist in `collections.abc` 
    - also in the `numbers` and `io` packages (and others)
* Python does not check for implementation of abstract methods at import time, only when you try to instantiate and object.
* When inheriting from ABCs, you typically has a set of abstract methods you must implement (more or less the public interface of the class) and often also inherit some ready-to-use concrete methods implemented in terms of aforementioned abstract methods.
* 16 abcs in collections.abc
    - top level: `Iterable`, `Container`, `Sized`, `Callable`, `Hashable`
    - 2nd level: `Iterator`, `Sequence`, `Mapping`, `Set`, `MappingView`
    - bottom levl: `MutableSequence`, `MutableMapping`, `MutableSet`, `ItemsView`, `ValueView`, `KeysView`
* abcs in `numbers`
    - In order of inheritance (first is topmost super): `Number` <- `Complex` <- `Real` <- `Rational` <- `Integer`

<br>
<hr>
Much of the power of ABCs it to create concrete methods based on public interface, that are inherited by any class that inherits from abc
<hr>
<br>

### Defining and Using an ABC
* define ABC by sublcassing `abc.ABC`
    - in older python this class not available, so had to use `metaclass=abc.ABCMeta`
    - and even older, where metclass not avaialble, had to set `__metaclass__` class attribute
* abstract methods are decorated by `@abstractmethod` and are typically empty except for docstring
    - this can instruct general behaviors, like which errors to raise in which cases
    - abstract method can actually have an implementation - sublcasses will still be forced to implement, but they can access abc implementation via super()
* also have `@abstractclassmethod`, `@abstractstaticmethod`, `@abstractproperty`
    - deprecated since python 3.3 in favor of stacking decorators
    - abstractmethod decorator must be aplied as innermost decorator
* Before abc existed, abstract methods used `NotImplementedError` to signal subclass was responsible for implementation
* sometimes the concrete methods reliance on public interface can be very expensive, so it behooves us to overrride
    - think of it as providing a backup

common idiom:
```
def __init__(self, iterable):
    self._balls = list(iterable)
```
* can pass any iterable
* can use list methods
* makes a copy not just a reference map

* crucial dynamic feature of goose typing: declaring virtual subclasses with the `register` method.
    - can register a class as a **virtual subclass** of an ABC, even if it doesnt in herit from it
    - when doing so, we promise that the calss faithfully implements the interface defined byt the ABC, and python believes us without checking; if we lie, we get caught by runtime exceptions.
    - benefit is that the virtual subclass will be recognized by `issubclass` and `isinstance` but WILL NOT INHERIT ANY METHODS OR ATTRIBUTES
* <font color='red'> Why not just inherit? </font>
* usage: decorate class with `@<abc_class_name>.register`
    - can also use as a regular function
    - this is actually how it is done most often - ie in collections.abc module source code
    ` Sequence.register(tuple)`

cool approach:
``` 
class TomoboList(list):
    def pick(self):
    ....
    
    
    load = list.extend
    
    def loaded(self):
    ...
```

load was defined without a def keyword

### hierarchy introspection
* two class attributes that allow introspection of a class hierarchy
    - `__subclasses__()` - returns list of immediate subclasses (not including virtual subclasses)
           * only lists descendants alive in memory (i.e they must be loaded)
    - `_abc_registry` -  data attribute, available only in ABCs. Bound to a WeakSet, with weak references to registered virtual subclasses.

Some ABCs have a `__subclasshook__(cls, C)` method that will recoginize another class as a virtual subclass even without having been declared as such - it does this by checking for special methods being defined.

### typed?
* srong vs weak typed:
    - python is strong typed (not a lot of implicit conversion of types)
* static vs dynamic typed:
    - python is dynamic typed - no type checking performed at compile time -all at runtime.